<a href="https://colab.research.google.com/github/Victor0vich/Denis/blob/main/%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_3_%D0%B4%D0%B5%D0%BA%D0%B0%D0%B1%D1%80%D1%8F_%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D0%BA%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D0%BD%D1%82_%D0%BD%D0%B0_Llama_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Примечание!!! В этом колабе используется база знаний, созданная на основе информации, взятой с сайта https://skyeng.ru/  (Школа английского языка Skyeng)


## Бaза знаний

>Размер базы - 94 страницы

Ссылка на базу:

https://docs.google.com/document/d/1aNcrmTAok2PCV5r6xPGswY8s7NKNlgC-mxyosEipjfo/edit?usp=sharing

# Схема агентов

                    

```

                       |Агент Отчета о потребностях|
                      ______________|_________________________
                     |                                        |
                    /\                                |Агент Презентации|
                   /  \                                       |        
                  /    \                              |Агент Стилизации 2|
|Агент Потребностей|  |Агент Выявления вопросов|              |
         |                         |                  |Агент Консультант|
         |                |Агент Консультант|                 
         |_________________________|                  
                    |                                         
           |Агент Стилизации 1|                       
                                                             
                                                      
                        

```

* `Агент Отчета о потребностях` - этот агент создает отчет о потребностях


* `Агент Потребностей` - этот агент формулирует вопросы для выявления потребностей клиента

* `Агент Выявления вопросов` - этот агент определяет клиент размышляет/отвечает или задает вопрос. Если размышляет/отвечает, то эти размышления переделываются в форму вопроса перед подачей в `Агент Консультант`. Если клиент задает вопрос, то этот вопрос без изменений подается в `Агент Консультант`

* `Агент Консультант` - этот агент формирует ответы на основании базы знаний УИИ

* `Агент Презентации` - этот агент формирует презентацию опираясь на отчет по потребностям

* `Агент Стилизации 1` - этот агент "причесывает" итоговый ответ после `Агент Консультант`, убирая лишнее

* `Агент Стилизации 2` - этот агент "причесывает" итоговый ответ в стиле продажи после `Агент Презентации`, убирая лишнее


In [ ]:
#@title Установка и импорт библиотек

!pip install tiktoken langchain==0.0.330 openai==0.28.1 faiss-cpu==1.7.3

import textwrap

# Модуль для работы с функциями, связанными с паролями
import getpass

# Модуль для загрузки файлов из Google Drive
import gdown


# Модуль для работы с файловой системой и операционной системой
import os

# Модуль для работы с регулярными выражениями
import re

# Библиотека для работы с API OpenAI
import openai

# Модуль для создания глубоких копий объектов
import copy

# Импортируем стандартный модуль time, который предоставляет функции для работы с временем
import time

# Модуль для работы с токенизацией текста
import tiktoken

# Импортируем загрузчик текстовых документов из langchain
from langchain.document_loaders import TextLoader

# Импортируем прогресс-бар для Jupyter Notebook
from tqdm import tqdm_notebook as tqdm

# Модуль для работы с сериализацией и десериализацией объектов
import pickle

# Импортируем модель OpenAI из langchain
from langchain.llms import OpenAI

# Импортируем класс Document из langchain для работы с документами
from langchain.docstore.document import Document

# Библиотека для выполнения HTTP-запросов
import requests

# Импортируем класс для создания эмбеддингов с помощью OpenAI из langchain
from langchain.embeddings.openai import OpenAIEmbeddings

# Импортируем класс FAISS для работы с векторными индексами из langchain
from langchain.vectorstores import FAISS

# Импортируем класс для рекурсивного разделения текста на фрагменты из langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Игнорирование предупреждений
import warnings
warnings.filterwarnings("ignore")

# Настройка логирования для модуля text_splitter из langchain
import logging
logging.getLogger("langchain.text_splitter").setLevel(logging.ERROR)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.143
    Uninstalling langsmith-0.1.143:
      Successfully uninstalled langsmith-0.1.143
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
  Attempting uninstall: langchain
    Found exis

In [ ]:
# Получение ключа API OpenAI от пользователя
# Функция getpass.getpass() запрашивает ввод пароля или ключа без отображения ввода на экране
openai_key = getpass.getpass("OpenAI API Key:")

# Установка ключа API в переменную окружения
os.environ["OPENAI_API_KEY"] = openai_key

# Установка ключа API для использования библиотекой openai
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
#@title Загружаем базу знаний

# Скачиваем файл с Google Drive по указанной ссылке и сохраняем его под именем 'faiss_index.pkl'
gdown.download('https://drive.google.com/uc?id=1XpnLrKLJSnH7gC_kBltSsaezP0ti4F55', 'faiss_index.pkl', quiet=False)

def load_db_from_file(filename):
    with open(filename, 'rb') as f:  # Открываем файл для чтения в бинарном режиме
        db_loaded = pickle.load(f)   # Загружаем объект из файла с помощью pickle
        print("FAISS index loaded from faiss_index.pkl")  # Выводим сообщение о успешной загрузке
    return db_loaded  # Возвращаем загруженный объект



Downloading...
From: https://drive.google.com/uc?id=1XpnLrKLJSnH7gC_kBltSsaezP0ti4F55
To: /content/faiss_index.pkl
100%|██████████| 1.01M/1.01M [00:00<00:00, 155MB/s]


In [ ]:
# Загрузка объекта db в переменную
db = load_db_from_file('faiss_index.pkl')

FAISS index loaded from faiss_index.pkl


In [ ]:
# @title Установка и импорт библиотеки Llama
!pip install tqdm requests

import torch

if torch.cuda.is_available():
    print('=========== GPU подключен! ===========')
    # !CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python==0.2.85
    !CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python[server]==0.2.85

else:
    print('=========== CPU подключен! ===========')
    !pip install -q --upgrade --force-reinstall llama-cpp-python==0.2.85 --no-cache-dir



=========== GPU подключен! ===========
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 44.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.85-cp310-cp310-linux_x86_64.whl size=255061975 sha256=77a443a3102919c8399aaffd341f1c6b35b4da4bac16070bb8f6525f41ae98f8
  Stored in directory: /root/.cache/pip/wheels/3f/e8/4e/29a754f9175ef52b6481cd75e3af4de38bf6dfa9c2972f75d4
Successfully built llama-cpp-python


In [ ]:
# Конфигурация модели Llama-3.1-70B
model_config = {
    "url": 'https://huggingface.co/lmstudio-community/Meta-Llama-3.1-70B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-70B-Instruct-Q4_K_M.gguf',
    "chat_format": None,  # Формат сообщений (None означает стандартный формат)
    "n_ctx": 8192,  # Максимальная длина контекста для модели
    "n_gpu_layers": 65  # Количество слоев, которые будут выполняться на GPU
}

In [ ]:
# Конфигурация модели Llama-3.1-8B
model_config =  {
      "url":'https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf',
      "chat_format": None,  # Формат сообщений (None означает стандартный формат)
      "n_ctx": 4096,  # Максимальная длина контекста для модели
      "n_gpu_layers": 41  # Количество слоев, которые будут выполняться на GPU
}

In [ ]:
# @title Функции для загрузки модели
import os  # Модуль для работы с файловой системой
from llama_cpp import Llama  # Импортируем класс для работы с Llama
from tqdm import tqdm  # Удобная библиотека для отображения прогресс-баров
import requests  # Библиотека для выполнения HTTP-запросов


# 1. Функция загрузки файла модели
def download_model(model_config):
    """
    Загружает файл модели, если он отсутствует на локальной машине.
    """
    url = model_config.get('url')  # Извлекаем URL для скачивания модели
    filename = url.split('/')[-1]  # Имя файла берется из последней части URL

    if not os.path.exists(filename):  # Проверяем, существует ли файл на диске
        response = requests.get(url, stream=True)  # Отправляем HTTP-запрос для скачивания файла
        total_size = int(response.headers.get('content-length', 0))  # Определяем общий размер файла
        block_size = 1024  # Размер блока (1 КБ) для чтения файла по частям
        t = tqdm(total=total_size, unit='iB', unit_scale=True)  # Инициализируем прогресс-бар
        with open(filename, 'wb') as file:  # Открываем файл для записи в бинарном режиме
            for data in response.iter_content(block_size):  # Считываем файл блоками
                t.update(len(data))  # Обновляем прогресс-бар
                file.write(data)  # Пишем данные в файл
        t.close()  # Закрываем прогресс-бар
        if total_size != 0 and t.n != total_size:  # Проверяем, полностью ли скачан файл
            raise ValueError("Ошибка при загрузке файла модели.")  # Если нет, выбрасываем исключение
    else:
        print(f"Файл модели {filename} уже существует.")  # Уведомляем, что файл уже есть
    return filename  # Возвращаем имя файла

# 2. Функция загрузки модели
def load_llama_model(filename, model_config):
    """
    Загружает модель Llama из указанного файла конфигурации.
    """
    llm = Llama(  # Инициализируем объект модели Llama
        model_path=filename,  # Путь к файлу модели
        chat_format=model_config['chat_format'],  # Формат сообщений
        n_ctx=model_config['n_ctx'],  # Максимальная длина контекста
        n_threads=16,  # Количество потоков для обработки (зависит от ресурсов системы)
        n_gpu_layers=model_config['n_gpu_layers'],  # Слои для выполнения на GPU
        f16_kv=True,  # Использование 16-битных ключей/значений для экономии памяти
        verbose=False  # Убираем лишнюю информацию из вывода
    )
    return llm  # Возвращаем загруженную модель


In [ ]:
# Скачиваем модель, если она еще не существует
filename = download_model(model_config)  # Загружаем модель

100%|██████████| 42.5G/42.5G [16:53<00:00, 42.0MiB/s]


In [ ]:
# Загружаем модель с помощью Llama
llm = load_llama_model(filename, model_config)  # Загружаем модель

## Агент Консультант для ответов на вопросы по базе знаний

In [ ]:
#@title Функция для получения ответа на вопрос на основе заданной темы и поискового индекса
def answer_kons(system, instruction, topic, search_index, summary_history, temp=1, verbose=0, k=8, model="gpt-3.5-turbo-16k"):
    """
    Функция для получения ответа на вопрос на основе заданной темы и поискового индекса.

    Параметры:
    system (str): Промт.
    instruction (str): Инструкция для пользователя.
    topic (str): Тема или вопрос, на который нужно ответить.
    search_index (object): Векторная база.
    temp (float): Температура для генерации ответов модели (по умолчанию 1).
    verbose (int): Флаг для включения/выключения подробного вывода (по умолчанию 0).
    k (int): Количество похожих документов для поиска (по умолчанию 8).
    model (str): Имя модели, используемой для генерации ответов (по умолчанию "gpt-3.5-turbo-16k").
    """

    # Поиск документов, похожих на заданную тему/вопрос
    docs = search_index.similarity_search(topic, k=k)

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n ========\n')

    # Создание содержимого сообщения, объединяя текст найденных документов
    message_content = re.sub(r'\r\n', ' ', '\n '.join([f'\n--------------------\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    # Если включен подробный вывод, напечатать содержимое сообщения
    if verbose: print('Чанки :\n ======================================== \n', message_content)

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instruction}.\n{message_content}.\n\nВопрос:\n{topic}\n\nХронология предыдущих сообщений диалога: {summary_history}\n\nОтвет:"}
    ]

    start_time = time.time()  # Начало отсчета времени

    # Создание запроса к модели OpenAI
    completion = openai.ChatCompletion.create(
        model=model,  # Имя модели
        messages=messages,  # Сообщения для модели
        temperature=temp  # Температура генерации
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n =========================================== ')


    # Получение ответа от модели
    answer = completion.choices[0].message.content
    #print()
    #print()
    #print("Полный ответ от GPT:")
    #print(completion)
    print('\n==================')
    print("Менеджер:")
    # Форматирование ответа для удобства чтения
    formatted_answer = textwrap.fill(answer, width=120)
    print(formatted_answer)
    #print(f"Количество использованных токенов на запрос: {completion['usage']['prompt_tokens']}")
    #print(f"Количество использованных токенов на ответ: {completion['usage']['completion_tokens']}")
    #print(f"Общее количество использованных токенов (вопрос-ответ): {completion['usage']['total_tokens']}")
    #print()
    #print('ЦЕНА запроса:', (3 * completion['usage']['prompt_tokens'] / 1000000) + (4 * completion['usage']['completion_tokens']/ 1000000), ' $')  # Вычисляем стоимость запроса
    print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer


def answer_kons_d(system, instruction, topic, search_index, summary_history, temp=1, verbose=0, k=8, model="gpt-3.5-turbo-16k"):
    """
    Функция для получения ответа на вопрос на основе заданной темы и поискового индекса.

    Параметры:
    system (str): Промт.
    instruction (str): Инструкция для пользователя.
    topic (str): Тема или вопрос, на который нужно ответить.
    search_index (object): Векторная база.
    temp (float): Температура для генерации ответов модели (по умолчанию 1).
    verbose (int): Флаг для включения/выключения подробного вывода (по умолчанию 0).
    k (int): Количество похожих документов для поиска (по умолчанию 8).
    model (str): Имя модели, используемой для генерации ответов (по умолчанию "gpt-3.5-turbo-16k").
    """

    # Поиск документов, похожих на заданную тему/вопрос
    docs = search_index.similarity_search(topic, k=k)

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n ========\n')

    # Создание содержимого сообщения, объединяя текст найденных документов
    message_content = re.sub(r'\r\n', ' ', '\n '.join([f'\n--------------------\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    # Если включен подробный вывод, напечатать содержимое сообщения
    if verbose: print('Чанки :\n ======================================== \n', message_content)

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instruction}.\n{message_content}.\n\nВопрос:\n{topic}\n\nХронология предыдущих сообщений диалога: {summary_history}\n\nОтвет:"}
    ]

    start_time = time.time()  # Начало отсчета времени

    # Создание запроса к модели OpenAI
    completion = openai.ChatCompletion.create(
        model=model,  # Имя модели
        messages=messages,  # Сообщения для модели
        temperature=temp  # Температура генерации
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n =========================================== ')


    # Получение ответа от модели
    answer = completion.choices[0].message.content
    #print()
    #print()
    #print("Полный ответ от GPT:")
    #print(completion)
    print('\n==================')
    print("Менеджер:")
    # Форматирование ответа для удобства чтения
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(formatted_answer)
    #print(f"Количество использованных токенов на запрос: {completion['usage']['prompt_tokens']}")
    #print(f"Количество использованных токенов на ответ: {completion['usage']['completion_tokens']}")
    #print(f"Общее количество использованных токенов (вопрос-ответ): {completion['usage']['total_tokens']}")
    #print()
    #print('ЦЕНА запроса:', (3 * completion['usage']['prompt_tokens'] / 1000000) + (4 * completion['usage']['completion_tokens']/ 1000000), ' $')  # Вычисляем стоимость запроса
    #print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer

def answer_kons_d_llama(model, system, instruction, topic, search_index, summary_history, temp=1, verbose=0, k=8, max_tokens=1000):


    # Поиск документов, похожих на заданную тему/вопрос
    docs = search_index.similarity_search(topic, k=k)


    # Создание содержимого сообщения, объединяя текст найденных документов
    message_content = re.sub(r'\r\n', ' ', '\n '.join([f'\n--------------------\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    # Если включен подробный вывод, напечатать содержимое сообщения
    if verbose: print('Чанки :\n ======================================== \n', message_content)

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instruction}.\n{message_content}.\n\nВопрос:\n{topic}\n\nХронология предыдущих сообщений диалога: {summary_history}\n\nОтвет:"}
    ]

    start_time = time.time()  # Начало отсчета времени

    # Создание запроса к модели OpenAI
    completion = model.create_chat_completion(  # Вызываем метод для генерации ответа
            max_tokens=max_tokens,  # Максимальное количество токенов в ответе
            temperature=temp,  # Температура генерации (0 = детерминированный ответ)
            messages=messages # Сообщения для модели
        )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n =========================================== ')


    # Получение ответа от модели
    answer = completion['choices'][0]['message']['content']
    #print()
    #print()
    #print("Полный ответ от GPT:")
    #print(completion)
    print('\n==================')
    print("Менеджер:")
    # Форматирование ответа для удобства чтения
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(formatted_answer)
    #print(f"Количество использованных токенов на запрос: {completion['usage']['prompt_tokens']}")
    #print(f"Количество использованных токенов на ответ: {completion['usage']['completion_tokens']}")
    #print(f"Общее количество использованных токенов (вопрос-ответ): {completion['usage']['total_tokens']}")
    #print()
    #print('ЦЕНА запроса:', (3 * completion['usage']['prompt_tokens'] / 1000000) + (4 * completion['usage']['completion_tokens']/ 1000000), ' $')  # Вычисляем стоимость запроса
    #print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer


def answer_kons_llama(model, system, instruction, topic, search_index, summary_history, temp=1, verbose=0, k=8, max_tokens=1000):

    # Поиск документов, похожих на заданную тему/вопрос
    docs = search_index.similarity_search(topic, k=k)

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n ========\n')

    # Создание содержимого сообщения, объединяя текст найденных документов
    message_content = re.sub(r'\r\n', ' ', '\n '.join([f'\n--------------------\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    # Если включен подробный вывод, напечатать содержимое сообщения
    if verbose: print('Чанки :\n ======================================== \n', message_content)

        # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instruction}.\n{message_content}.\n\nВопрос:\n{topic}\n\nХронология предыдущих сообщений диалога: {summary_history}\n\nОтвет:"}
    ]


    start_time = time.time()  # Начало отсчета времени

    completion = model.create_chat_completion(  # Вызываем метод для генерации ответа
            max_tokens=max_tokens,  # Максимальное количество токенов в ответе
            temperature=temp,  # Температура генерации (0 = детерминированный ответ)
            messages=messages # Сообщения для модели
        )
    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion['choices'][0]['message']['content']
    formatted_answer = textwrap.fill(answer, width=120)

    print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return formatted_answer  # Возвращаем текст ответа

Промт и инструкция

In [ ]:
# Подгружаем промт
system = ''' # Ты профессиональный помощник в чате компании


## 1. Общие обязанности и цели - Компания продает курсы по английскому языку. У компании есть большой документ со всеми материалами
о продуктах компании. - Твоя обязанность: Дать Клиенту краткий корректный ответ на русском языке в чате, опираясь на отрывки
из этого документа. - Твоя цель: Отвечать максимально кратко и точно по документу, не придумывать ничего от себя.


## 2. Ограничения в общении - Запрещено общаться на стороннюю тему. Если Клиент задает стороннюю тему, спрашивает не
по теме английского языка, не по материалам и продуктам Компании, ты категорически отказываешься
отвечать.


## 3. Секретность документа
- Запрещено упоминать в ответе, что ты анализировал отрывки документов и брал оттуда информацию.

'''

# Инструкция для модели о том, как использовать контекст для ответа на вопрос
instruction = '''Проанализируй предыдущий диалог чтобы написать свой ответ последовательным и логичным.
Категорически запрещено повторяться и здороваться.
Используйте следующие фрагменты контекста, чтобы ответить на вопрос в конце.
Стилистика ответа должна быть поддерживающей беседу в контексте важности и полезности изучения английского языка.
Если вы не знаете ответа, просто скажите, что не знаете, не пытайтесь придумывать ответ.
Тебе запрещено продавать, предлагать курсы. Запрещено спрашивать клиента что его еще интересует.'''

### Модель llama-3.1-8b

Параметры для запроса

In [ ]:
# Задаем температуру генерации ответа
# Температура 0 означает, что модель будет давать более детерминированные ответы, строго опираясь на предоставленный контекст
temperature = 0

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose = 0

# Имя модели, используемой для генерации ответов
model = llm

max_tokens=1000 # Количество токенов на ответ

# История диалога
history = [""]

Запрос с чанками

In [ ]:
topic = "Есть ли пробные уроки?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Чанки :
 
--------------------
2. 96 + 3 урока  
1 350 ₽ за урок  




3. 64 + 3 урока  
1 550 ₽ за урок  




4. 32 + 3 урока  
1 850 ₽ за урок  




5. 16 + 2 урока  
1 990 ₽ за урок  




6. 8 + 1 урок  
2 350 ₽ за урок  




7. 4 + 1 урок  
2 690 ₽ за урок  






## Что такое пробный урок?


Это бесплатное занятие, которое позволит вам освоиться. Первая часть проходит индивидуально: можно «пощупать» платформу, пройти небольшой тест, определить уровень языка и подобрать программу. После к вам подключится методист, чтобы ответить на все вопросы, рассказать о процессе обучения и помочь сформировать расписание.


## Я могу получить бесплатный пробный урок только после оплаты?


## Что нужно, чтобы начать обучение онлайн?


## Не получается прийти на урок, что делать?


## Мало свободного времени, боюсь, не буду успевать заниматься


## Нужно ли что-то из канцелярии для пробного урока?


## Сколько длится один урок?


## Экспресс-курсы — это не вредно?


## Что еще входит в цену курса,

Запрос 1

In [ ]:
topic = "Что нужно, чтобы начать изучение английского онлайн с Skyeng?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 1.69 секунд
Ответ: Чтобы начать изучение английского онлайн с Skyeng, вам не нужно ничего, кроме устройства с микрофоном, камерой и
доступом к сайту Skyeng.


Запрос 2

In [ ]:
topic = "Какова продолжительность урока английского в Skyeng?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 1.64 секунд
Ответ: Одно занятие в Skyeng длится 50 минут.


Запрос 3

In [ ]:
topic = "Как проходит индивидуальное занятие с репетитором?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 7.80 секунд
Ответ: Давайте разберемся в вопросе. Вы хотите узнать, как проходит индивидуальное занятие с репетитором.   В нашем курсе
каждый студент получает индивидуальную стратегию изучения английского, разработанную нашими методистами. Это означает,
что репетитор подберет для вас подходящую программу и учебные пособия, исходя из ваших ресурсов, целей и графика.
Также вы сможете участвовать в разговорных клубах и получать методические рекомендации от специалиста, который
отслеживает ваш учебный процесс.   Это все, что я могу сказать на этот вопрос.


Запрос 4

In [ ]:
topic = "В чем преимущество изучения английского именно в Skyeng?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 4.19 секунд
Ответ: В Skyeng вы сможете изучать английский язык с помощью коммуникативной методики, которая позволяет вам начать говорить на
английском уже с первого занятия и практиковаться в разговорной речи большую часть времени. Это поможет вам получить
реальные навыки общения и переступить языковой барьер.


Запрос 5

In [ ]:
topic = "Если я вообще не знаю английского, есть смысл начинать его учить?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 3.88 секунд
Ответ: Да, смысл начинать учить английский язык, даже если вы не знаете его совсем. Это поможет вам свободно общаться с людьми,
читать и писать на английском, что может быть полезно в личной и профессиональной жизни.


### Модель llama-3.1-70b

Параметры для запроса

In [ ]:
# Задаем температуру генерации ответа
# Температура 0 означает, что модель будет давать более детерминированные ответы, строго опираясь на предоставленный контекст
temperature = 0

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose = 0

# Имя модели, используемой для генерации ответов
model = llm

max_tokens=1000 # Количество токенов на ответ

# История диалога
history = [""]

Запрос 1

In [ ]:
topic = "Что нужно, чтобы начать изучение английского онлайн с Skyeng?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 15.60 секунд
Ответ: Чтобы начать обучение, вам понадобится устройство с микрофоном, камерой и доступом к сайту Skyeng.


Запрос 2

In [ ]:
topic = "Какова продолжительность урока английского в Skyeng?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 8.75 секунд
Ответ: Одно занятие в Skyeng длится 50 минут.


Запрос 3

In [ ]:
topic = "Как проходит индивидуальное занятие с репетитором?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 16.65 секунд
Ответ: Для эффективного обучения наши методисты составят программу и подберут учебные пособия, исходя из ваших ресурсов, целей
и графика.


Запрос 4

In [ ]:
topic = "В чем преимущество изучения английского именно в Skyeng?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 22.29 секунд
Ответ: В Skyeng вы будете изучать правила только 20% времени, а остальную часть урока посвятите разговорной практике. Это
поможет привыкнуть к общению, живой речи и научиться разговаривать свободно и без стеснения.


Запрос 5

In [ ]:
topic = "Если я вообще не знаю английского, есть смысл начинать его учить?"
answer = answer_kons_llama(model, system, instruction, topic, db, history, temperature, verbose, relevant_chanks, max_tokens)
print("Ответ:", answer)

Время ответа: 34.65 секунд
Ответ: Да, есть смысл начинать учить английский, даже если вы не знаете его совсем. Мы предлагаем программу обучения
английскому с нуля, которая включает 44 онлайн-урока и 9 тематических модулей. Вы сможете начать читать и писать на
английском, общаться с иностранцами на простые темы и познакомиться с основами языка.


### Модель GPT-4o-mini

Параметры запроса

In [ ]:
# Задаем температуру генерации ответа
# Температура 0 означает, что модель будет давать более детерминированные ответы, строго опираясь на предоставленный контекст
temperature = 0

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose = 0

# Имя модели, используемой для генерации ответов
model = "gpt-4o-mini"

# История диалога

history = [""]

Запрос 1

In [ ]:
topic = "Что нужно, чтобы начать изучение английского онлайн с Skyeng?"
answer_kons(system, instruction, topic, db, history, temperature, verbose, relevant_chanks, model);


Менеджер:
Чтобы начать изучение английского онлайн с Skyeng, вам нужно лишь устройство с микрофоном, камерой и доступом к сайту.
Это достаточно для начала обучения.
Время ответа: 0.95 секунд


Запрос 2

In [ ]:
topic = "Какова продолжительность урока английского в Skyeng?"
answer_kons(system, instruction, topic, db, history, temperature, verbose, relevant_chanks, model);


Менеджер:
Одно занятие в Skyeng длится 50 минут.
Время ответа: 0.73 секунд


Запрос 3

In [ ]:
topic = "Как проходит индивидуальное занятие с репетитором?"
answer_kons(system, instruction, topic, db, history, temperature, verbose, relevant_chanks, model);


Менеджер:
Индивидуальное занятие с репетитором проходит в комфортном для вас темпе. Преподаватели, прошедшие дополнительное
обучение, помогут построить программу, учитывая ваши материалы и цели. Вы можете общаться с преподавателем вне занятий,
а также вносить изменения в расписание по мере необходимости.
Время ответа: 1.64 секунд


Запрос 4

In [ ]:
topic = "В чем преимущество изучения английского именно в Skyeng?"
answer_kons(system, instruction, topic, db, history, temperature, verbose, relevant_chanks, model);


Менеджер:
Преимущество изучения английского в Skyeng заключается в использовании коммуникативной методики, которая позволяет
студентам говорить на английском уже с первого занятия. 80% времени на уроках посвящено разговорной практике, что
помогает преодолеть языковой барьер и развить реальные навыки общения. Кроме того, опытные преподаватели, прошедшие
строгий отбор, обеспечивают качественное обучение, а гибкая структура курсов позволяет адаптировать программу под
индивидуальные цели ученика.
Время ответа: 2.66 секунд


Запрос 5

In [ ]:
topic = "Если я вообще не знаю английского, есть смысл начинать его учить?"
answer_kons(system, instruction, topic, db, history, temperature, verbose, relevant_chanks, model);


Менеджер:
Да, есть смысл начинать учить английский язык с нуля. Это поможет вам расширить словарный запас, разобраться в
грамматике и научиться понимать живую речь на слух. Вы сможете свободно общаться с людьми, читать и писать на
английском. Начать изучение проще, чем кажется, и можно делать это в удобном для вас темпе.
Время ответа: 2.11 секунд


## Агент Отчета о потребностях

In [ ]:
#@title Функция для маршрутизации вопроса пользователя
def user_question_router(system, instructions, topic, temp=0, verbose=0, model='gpt-3.5-turbo-1106'):
    #if verbose: print('\n==================\n')
    #if verbose: print('Саммари диалога:\n==================\n',
                      #topic)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                      \n\nВопрос менеджера и ответ клиента:\n{topic}.
                                      \n\nОтвет: '''}
    ]
    start_time = time.time()  # Начало отсчета времени
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )
    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion.choices[0].message.content
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    #print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")
    if verbose: print('\n==================')
    if verbose: print(f'Агент Отчета о потребностях:\n',
                      f'{formatted_answer}')
    return answer

# Функция суммарной потребности
def user_potr(potr_history, temp=0, verbose=0, model= "gpt-4o"):
    resulting_string = ", ".join(potr_history)
    completion_prez = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": """Вы лучший менеджер по продажам.
Вы понимаете какие потребности клиента надо выявить, чтобы полностью понять желания и боли клиента,
которые можно удовлетворить при помощи обучения английскому языку.
Вы знаете, что важно выявить есть ли у клиента такие потребности: для работы, учебы, путешествий, личного развития и т.д."""},
        {"role": "user", "content": f''' На основании Отчета по потребностям отпредели основную потребность клиента. В ответе четко сформулируй только эту потребность в виде темы для статьи.

         Отчет по потребностям: {resulting_string}'''}],
        temperature=0
    )
    answer_prez = completion_prez.choices[0].message.content
    #print(answer_prez)
    return answer_prez



def user_question_router_llama(model, system, instructions, topic, temp=0, verbose=0, max_tokens=1000):
    #if verbose: print('\n==================\n')
    #if verbose: print('Саммари диалога:\n==================\n',
                      #topic)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                      \n\nВопрос менеджера и ответ клиента:\n{topic}.
                                      \n\nОтвет: '''}
    ]

    start_time = time.time()  # Начало отсчета времени

    completion = model.create_chat_completion(  # Вызываем метод для генерации ответа
            max_tokens=max_tokens,  # Максимальное количество токенов в ответе
            temperature=temp,  # Температура генерации (0 = детерминированный ответ)
            messages=messages # Сообщения для модели
        )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion['choices'][0]['message']['content']
    formatted_answer = textwrap.fill(answer, width=120)

    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")


    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    #print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print('\n==================')
    if verbose: print(f'Агент Отчета о потребностях:\n',
                      f'{formatted_answer}')
    return answer

# Функция суммарной потребности
def user_potr_llama(model, potr_history, temp=0, verbose=0, max_tokens=1000):
    resulting_string = ", ".join(potr_history)

    messages=[
        {"role": "system", "content": """Вы лучший менеджер по продажам.
Вы понимаете какие потребности клиента надо выявить, чтобы полностью понять желания и боли клиента,
которые можно удовлетворить при помощи обучения английскому языку.
Вы знаете, что важно выявить есть ли у клиента такие потребности: для работы, учебы, путешествий, личного развития и т.д."""},
        {"role": "user", "content": f''' На основании Отчета по потребностям отпредели основную потребность клиента. В ответе четко сформулируй только эту потребность в виде темы для статьи.

         Отчет по потребностям: {resulting_string}'''}]

    completion_prez = model.create_chat_completion(  # Вызываем метод для генерации ответа
            max_tokens=max_tokens,  # Максимальное количество токенов в ответе
            temperature=temp,  # Температура генерации (0 = детерминированный ответ)
            messages=messages # Сообщения для модели
        )


    answer_prez = completion_prez['choices'][0]['message']['content']
    #print(answer_prez)
    return answer_prez

Промт и инструкция

In [ ]:
system_prompt_router = '''
Ты лучший специалист отдела продаж. Ты продаешь курсы обучения английскому языку.
Ты знаешь, что Потребность - это то, что клиент хочет или что ему нравится,
и что повлияет на приобретение им курсов обучения.
Ты очень хорошо умеешь выявлять в Вопросе менеджера и ответе клиента потребности клиента.
Ты всегда очень строго следуешь порядку отчета.
'''
instructions_router = '''
Давай действовать последовательно:
Проанализируй Вопрос менеджера и ответ клиента, выяви какие высказаны потребности клиента.
Ничего не придумывай от себя.
Порядок отчета: предоставь только название потребности с формулировкой клиента через двоеточие.'''


### Модель llama-3.1-8b

Параметры запроса

In [ ]:
model_router = llm # Имя модели, используемой для генерации ответов

temperature_router = 0 # Задаем температуру генерации ответа

verbose_router = 1 # Флаг для включения/выключения вывода ответа модели

max_tokens=1000 # Количество токенов на ответ

Запрос 1

In [ ]:
topic = "Какие есть уроки для детей"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 0.94 секунд

Агент Отчета о потребностях:
 Уроки для детей: есть уроки для детей от 5 лет.


Запрос 2

In [ ]:
topic = "Я хочу уехать за границу"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 0.49 секунд

Агент Отчета о потребностях:
 Я хочу уехать за границу.


Запрос 3

In [ ]:
topic = "Мне нужно выучить английский за год"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 0.47 секунд

Агент Отчета о потребностях:
 Выучить английский за год.


Запрос 4

In [ ]:
topic = "Я что-то хочу, но не знаю что"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 0.81 секунд

Агент Отчета о потребностях:
 Навык обучения английскому языку для улучшения общения.


Запрос 5

In [ ]:
topic = "Не хочу платить деньги за обучение"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 0.37 секунд

Агент Отчета о потребностях:
 Нет денег на обучение.


### Модель llama-3.1-70b

Параметры запроса

In [ ]:
model_router = llm # Имя модели, используемой для генерации ответов

temperature_router = 0 # Задаем температуру генерации ответа

verbose_router = 1 # Флаг для включения/выключения вывода ответа модели

max_tokens=1000 # Количество токенов на ответ

Запрос 1

In [ ]:
topic = "Какие есть уроки для детей"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 4.50 секунд

Агент Отчета о потребностях:
 Уроки для детей: для детей.


Запрос 2

In [ ]:
topic = "Я хочу уехать за границу"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 11.42 секунд

Агент Отчета о потребностях:
 Потребность: Уехать за границу: Я хочу уехать за границу.


Запрос 3

In [ ]:
topic = "Мне нужно выучить английский за год"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 6.25 секунд

Агент Отчета о потребностях:
 Срочность: выучить английский за год.


Запрос 4

In [ ]:
topic = "Я что-то хочу, но не знаю что"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 8.34 секунд

Агент Отчета о потребностях:
 Нежелание тратить время на поиск информации: "Я что-то хочу, но не знаю что."


Запрос 5

In [ ]:
topic = "Не хочу платить деньги за обучение"
user_question_router_llama(model_router, system_prompt_router, instructions_router, topic, temperature_router, verbose_router, max_tokens);

Время ответа: 6.61 секунд

Агент Отчета о потребностях:
 Экономия средств: не хочу платить деньги за обучение.


### Модель GPT-4o-mini

Параметры запроса

In [ ]:
model_router = 'gpt-4o-mini' # Имя модели, используемой для генерации ответов

temperature_router = 0 # Задаем температуру генерации ответа

verbose_router = 1 # Флаг для включения/выключения вывода ответа модели

Запрос 1

In [ ]:
topic = "Какие есть уроки для детей"
user_question_router(system_prompt_router, instructions_router, topic, temperature_router, verbose_router, model_router);

Время ответа: 1.59 секунд

Агент Отчета о потребностях:
 Потребность: Уроки для детей: "Какие есть уроки для детей?"


Запрос 2

In [ ]:
topic = "Я хочу уехать за границу"
user_question_router(system_prompt_router, instructions_router, topic, temperature_router, verbose_router, model_router);

Время ответа: 0.54 секунд

Агент Отчета о потребностях:
 Потребность: Желание эмигрировать: "Я хочу уехать за границу."


Запрос 3

In [ ]:
topic = "Мне нужно выучить английский за год"
user_question_router(system_prompt_router, instructions_router, topic, temperature_router, verbose_router, model_router);

Время ответа: 0.65 секунд

Агент Отчета о потребностях:
 Потребность: "Мне нужно выучить английский за год."


Запрос 4

In [ ]:
topic = "Я что-то хочу, но не знаю что"
user_question_router(system_prompt_router, instructions_router, topic, temperature_router, verbose_router, model_router);

Время ответа: 0.58 секунд

Агент Отчета о потребностях:
 Неопределенность в потребностях: "Я что-то хочу, но не знаю что."


Запрос 5

In [ ]:
topic = "Не хочу платить деньги за обучение"
user_question_router(system_prompt_router, instructions_router, topic, temperature_router, verbose_router, model_router);

Время ответа: 0.47 секунд

Агент Отчета о потребностях:
 Потребность: Необходимость в бесплатном обучении.


## Агент Потребностей

In [ ]:
#@title Функция для генерации вопроса пользователю на основе истории чата и текущего вопроса
# Функция для генерации отчета по истории чата
def generate_summary_report(history_chat):
    report = []
    for i, message in enumerate(history_chat, start=1):
        report.append(f"{i}. {message}")  # Нумеруем и добавляем каждое сообщение в отчет
    return "\n".join(report)  # Объединяем все сообщения в одну строку


# Функция для генерации вопроса пользователю на основе истории чата и текущего вопроса
def spez_user_question(system, instructions, needs, history_chat, temp=0, verbose=0, model="gpt-3.5-turbo-16k"):
    summary_history = generate_summary_report(history_chat)  # Генерация отчета по истории чата
    messages = [
        {"role": "system", "content": system},  # Добавляем системное сообщение
        {"role": "user", "content": f'''{instructions}

         Список потребностей: {needs}

         Хронология предыдущих сообщений диалога: {summary_history}'''}  # Добавляем инструкции и контекст
    ]

    # if verbose: print('\n==================\n')
    # if verbose: print(f'Вопрос клиента:', question)
    # if verbose: print('Саммари диалога:\n==================\n',
    #                      summary_history)
    # print("messages", messages)
    start_time = time.time()  # Начало отсчета времени

    # Запрос к модели OpenAI для генерации ответа
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )
    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # Извлекаем ответ из ответа модели
    answer = completion.choices[0].message.content
    try:
        # Пытаемся разделить ответ по ': ' и взять вторую часть
        answer = answer.split(': ')[1] + ' '
    except:
        # Если не удалось, используем оригинальный ответ
        answer = answer
    # Убираем возможные служебные символы в начале строки
    answer = answer.lstrip('#3')

    # if verbose: print(f'\n==================')
    # if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")
    if verbose: print('\n==================')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(f'Вопрос от Агента Потребностей:\n', f'{formatted_answer}')

    return answer  # Возвращаем ответ


def spez_user_question_llama(model, system, instructions, needs, history_chat, temp=0, verbose=0, max_tokens=1000):
    summary_history = generate_summary_report(history_chat)  # Генерация отчета по истории чата
    messages = [
        {"role": "system", "content": system},  # Добавляем системное сообщение
        {"role": "user", "content": f'''{instructions}

         Список потребностей: {needs}

         Хронология предыдущих сообщений диалога: {summary_history}'''}  # Добавляем инструкции и контекст
    ]

    # if verbose: print('\n==================\n')
    # if verbose: print(f'Вопрос клиента:', question)
    # if verbose: print('Саммари диалога:\n==================\n',
    #                      summary_history)
    # print("messages", messages)

    start_time = time.time()  # Начало отсчета времени

    completion = model.create_chat_completion(  # Вызываем метод для генерации ответа
            max_tokens=max_tokens,  # Максимальное количество токенов в ответе
            temperature=temp,  # Температура генерации (0 = детерминированный ответ)
            messages=messages # Сообщения для модели
        )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени


    # Извлекаем ответ из ответа модели
    answer = completion['choices'][0]['message']['content']
    #try:
        # Пытаемся разделить ответ по ': ' и взять вторую часть
        #answer = answer.split(': ')[1] + ' '
    #except:
        # Если не удалось, используем оригинальный ответ
        #answer = answer
    # Убираем возможные служебные символы в начале строки
    #answer = answer.lstrip('#3')

    # Удаляем переносы строк и лишние пробелы
    #answer0 = answer.replace('\n', ' ').strip()

    # Извлечение текста после последнего символа :
    #answer1 = answer0.rpartition(':')[-1].strip()

    # Убираем кавычки в начале и конце, если они есть
    #answer1 = answer1.strip('"')

    # Удаляем переносы строк и лишние пробелы
    text1 = answer.replace('\n', ' ').strip()

    # Извлекаем последнее предложение
    last_sentence1 = re.findall(r'([^.!?]*[.!?])', text1)[-1].strip()

    # Проверяем наличие двоеточия
    if ':' in last_sentence1:
        # Берём только часть после двоеточия и удаляем кавычки
        answer1 = last_sentence1.split(':', 1)[-1].strip().strip('"')
    else:
        # Если двоеточия нет, удаляем кавычки из всего предложения
        answer1 = last_sentence1.strip('"')






    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")

    # if verbose: print(f'\n==================')
    # if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    if verbose: print('\n==================')
    formatted_answer = textwrap.fill(answer1, width=120)
    if verbose: print(f'Вопрос от Агента Потребностей:\n', f'{formatted_answer}')

    return answer1  # Возвращаем ответ

Промт и инструкция

In [ ]:
# Системное сообщение для модели, задающее её поведение и контекст работы
system_prompt_potr = '''
Вы лучший менеджер по продажам. Вы работаете в компании,
которая продает курсы обучения английскому языку.
Вы очень хорошо знаете, какие курсы и программы обучения предоставляет компания.
Вы понимаете какие потребности клиента надо выявить, чтобы полностью понять желания и боли клиента,
которые можно удовлетворить при помощи обучения английскому языку.
Вы знаете, что важно выявить есть ли у клиента такие потребности: в повышении уровня знаний, для работы, для собеседований, для переезда,
для учебы, для подготовки к ЕГЭ, для детей, для путешествий, личного развития и т.д.
Вы понимаете, что нужно аккуратно и ненавязчиво выявить несколько разных потребностей и составляете свой ответ с этой целью.
Ваша задача: сформулировать вопрос клиенту, который поможет качественно выявить его потребности.
Вы всегда очень строго следуете порядку отчета.
'''

# Инструкции для модели, описывающие шаги, которые она должна выполнить
instructions__potr = '''
Давайте действовать по шагам:
#Шаг1: Проанализируйте диалог и Список потребностей;
#Шаг2: Предположите одну другую потребность, которой нет среди выявленных потребностей в Шаг1;
#Шаг3: Учитывая, что выявление потребностей должно провоцировать дальнейший диалог и заинтересовывать клиента,
для этой потребности из Шаг2 напишите один вопрос для качественного выявления этой потребности. Ничего, кроме вопроса не пишите и не объясняйте.
Порядок отчета: В свой ответ включите только один вопрос из Шаг3, ничего кроме вопроса выводить не нужно.
'''

### Модель llama-3.1-8b

Параметры запроса

In [ ]:
temp = 0.1  # Температура генерации ответов модели, влияет на креативность и разнообразие ответов

verbose = 1 # Флаг для включения/выключения вывода ответа модели

model_potr = llm # Имя модели, используемой для генерации ответов

max_tokens=1000 # Количество токенов на ответ

needs = [""] # Отчет о потребностях

Запрос 1

In [ ]:
history_chat = [
    "Менеджер: Здравствуйте, я нейро-консультант компании по изучению английского языка. Чем я могу Вам помочь?",
    "Клиент: Мне нужно изучить английский язык для работы."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 8.89 секунд

Вопрос от Агента Потребностей:
 Вы планируете использовать английский язык в конкретной профессии или сфере деятельности?


Запрос 2

In [ ]:
history_chat = [
    "Менеджер: Здравствуйте, я нейро-консультант компании по изучению английского языка. Чем я могу Вам помочь?",
    "Клиент: Я хочу подтянуть свой уровень английского для путешествий."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 9.03 секунд

Вопрос от Агента Потребностей:
 Вы планируете использовать английский язык в своей профессиональной деятельности в ближайшем будущем?


Запрос 3

In [ ]:
history_chat = [
    "Менеджер: Добрый день! Чем я могу помочь Вам в изучении английского языка?",
    "Клиент: Мне нужно подготовиться к ЕГЭ по английскому."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 7.80 секунд

Вопрос от Агента Потребностей:
 Как вы видите свою будущую роль в международном сообществе, и как вы думаете, может помочь вам английский язык?


Запрос 4

In [ ]:
history_chat = [
    "Менеджер: Здравствуйте! Какой цели Вы хотите достичь, изучая английский язык?",
    "Клиент: Хочу изучить английский для работы в международной компании."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 7.67 секунд

Вопрос от Агента Потребностей:
 Вы когда-нибудь думали о путешествии в страну, где английский язык является официальным или широко распространенным?


Запрос 5

In [ ]:
history_chat = [
    "Менеджер: Добрый день! Расскажите, для каких целей Вы планируете изучать английский язык?",
    "Клиент: Мне нужен английский для переезда в другую страну."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 1.46 секунд

Вопрос от Агента Потребностей:
 Для переезда в другую страну, Вы планируете занимать какую-либо конкретную профессию или работать на себя?


### Модель llama-3.1-70b

Параметры запроса

In [ ]:
temp = 0.1  # Температура генерации ответов модели, влияет на креативность и разнообразие ответов

verbose = 1 # Флаг для включения/выключения вывода ответа модели

model_potr = llm # Имя модели, используемой для генерации ответов

max_tokens=1000 # Количество токенов на ответ

needs = [""] # Отчет о потребностях

Запрос 1

In [ ]:
history_chat = [
    "Менеджер: Здравствуйте, я нейро-консультант компании по изучению английского языка. Чем я могу Вам помочь?",
    "Клиент: Мне нужно изучить английский язык для работы."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 12.16 секунд

Вопрос от Агента Потребностей:
 Вас интересует возможность применения английского языка в конкретной отрасли или профессии, или же Вам важен общий
деловой английский?


Запрос 2

In [ ]:
history_chat = [
    "Менеджер: Здравствуйте, я нейро-консультант компании по изучению английского языка. Чем я могу Вам помочь?",
    "Клиент: Я хочу подтянуть свой уровень английского для путешествий."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 9.67 секунд

Вопрос от Агента Потребностей:
 Вы планируете часто путешествовать за границу или это скорее разовая поездка?


Запрос 3

In [ ]:
history_chat = [
    "Менеджер: Добрый день! Чем я могу помочь Вам в изучении английского языка?",
    "Клиент: Мне нужно подготовиться к ЕГЭ по английскому."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 6.78 секунд

Вопрос от Агента Потребностей:
 Вам нужно улучшить только устный или письменный английский, или оба?


Запрос 4

In [ ]:
history_chat = [
    "Менеджер: Здравствуйте! Какой цели Вы хотите достичь, изучая английский язык?",
    "Клиент: Хочу изучить английский для работы в международной компании."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 13.42 секунд

Вопрос от Агента Потребностей:
 Планируете ли Вы в ближайшее время участвовать в международных бизнес-встречах или переговорах, где свободное владение
английским языком будет иметь решающее значение?


Запрос 5

In [ ]:
history_chat = [
    "Менеджер: Добрый день! Расскажите, для каких целей Вы планируете изучать английский язык?",
    "Клиент: Мне нужен английский для переезда в другую страну."
]
spez_user_question_llama(model_potr, system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, max_tokens);

Время ответа: 14.14 секунд

Вопрос от Агента Потребностей:
 Планируете ли Вы работать в стране, куда собираетесь переехать, и если да, то насколько важен для Вас английский язык
для карьерного роста?


### Модель GPT-4o-mini

Параметры запроса

In [ ]:
temp = 0.1  # Температура генерации ответов модели, влияет на креативность и разнообразие ответов

verbose = 1 # Флаг для включения/выключения вывода ответа модели

model_potr = 'gpt-4o-mini' # Имя модели, используемой для генерации ответов

needs = [""] # Отчет о потребностях

Запрос 1

In [ ]:
history_chat = [
    "Менеджер: Здравствуйте, я нейро-консультант компании по изучению английского языка. Чем я могу Вам помочь?",
    "Клиент: Мне нужно изучить английский язык для работы."
]
spez_user_question(system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, model_potr);

Время ответа: 0.50 секунд

Вопрос от Агента Потребностей:
 Как вы планируете использовать английский язык в своей работе?


Запрос 2

In [ ]:
history_chat = [
    "Менеджер: Здравствуйте, я нейро-консультант компании по изучению английского языка. Чем я могу Вам помочь?",
    "Клиент: Я хочу подтянуть свой уровень английского для путешествий."
]
spez_user_question(system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, model_potr);

Время ответа: 0.60 секунд

Вопрос от Агента Потребностей:
 Как часто вы планируете путешествовать, и какие страны вас интересуют для изучения английского в контексте общения с
местными жителями?


Запрос 3

In [ ]:
history_chat = [
    "Менеджер: Добрый день! Чем я могу помочь Вам в изучении английского языка?",
    "Клиент: Мне нужно подготовиться к ЕГЭ по английскому."
]
spez_user_question(system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, model_potr);

Время ответа: 0.49 секунд

Вопрос от Агента Потребностей:
 Как вы планируете использовать свои знания английского языка после успешной сдачи ЕГЭ?


Запрос 4

In [ ]:
history_chat = [
    "Менеджер: Здравствуйте! Какой цели Вы хотите достичь, изучая английский язык?",
    "Клиент: Хочу изучить английский для работы в международной компании."
]
spez_user_question(system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, model_potr);

Время ответа: 0.67 секунд

Вопрос от Агента Потребностей:
 Какой именно аспект работы в международной компании требует от вас знания английского языка?


Запрос 5

In [ ]:
history_chat = [
    "Менеджер: Добрый день! Расскажите, для каких целей Вы планируете изучать английский язык?",
    "Клиент: Мне нужен английский для переезда в другую страну."
]
spez_user_question(system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, model_potr);

Время ответа: 0.60 секунд

Вопрос от Агента Потребностей:
 Какой именно аспект жизни за границей вас больше всего беспокоит, и как знание английского языка может помочь вам в
этом?


## Агент Выявления вопросов

In [ ]:
#@title Функция для выявления вопросов
def user_question(system, instructions, topic, temp=0, verbose=0, model='gpt-3.5-turbo-1106'):

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                      \n\nСообщение клиента:\n{topic}
                                      \n\nОтвет: '''}
    ]
    start_time = time.time()  # Начало отсчета времени
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )
    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion.choices[0].message.content
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")
    if verbose: print('\n==================')
    if verbose: print(f'Ответ Агента по выявлению вопросов:\n',
                      f'{formatted_answer}')
    #print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    return answer


def user_question_llama(model, system, instructions, topic, temp=0, verbose=0, max_tokens=1000):

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                      \n\nСообщение клиента:\n{topic}
                                      \n\nОтвет: '''}
    ]
    start_time = time.time()  # Начало отсчета времени

    completion = model.create_chat_completion(  # Вызываем метод для генерации ответа
            max_tokens=max_tokens,  # Максимальное количество токенов в ответе
            temperature=temp,  # Температура генерации (0 = детерминированный ответ)
            messages=messages # Сообщения для модели
        )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени


    # Извлекаем ответ из ответа модели
    answer = completion['choices'][0]['message']['content']
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")
    if verbose: print('\n==================')
    if verbose: print(f'Ответ Агента по выявлению вопросов:\n',
                      f'{formatted_answer}')
    #print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    return answer

Промт и инструкция

In [ ]:
system_prompt_question ='''
Ты идеально справляешься со своей задачей: ты отлично определяешь вопрос клиента.
Проверь есть ли в сообщении клиента какой-нибудь вопрос.
Если вопрос есть, то напиши сообщение клиента без изменений.
Если вопроса нет, а есть общие рассуждения, то сформулируй на основании сообщения клиента вопрос.

Ты всегда очень строго следуешь требованиям к порядку отчета.'''
instructions_question = '''
Пожалуйста, будем действовать по шагам:
#Шаг1: проанализируйте Сообщение клиента чтобы быть в контексте;
#Шаг2: опираясь на анализ Шаг1 сформулируй Вопрос клиента.
Отвечай, пожалуйста, точно, и ничего не придумывай от себя.
Порядок отчета: напиши только вопрос клиента.'''

### Модель llama-3.1-8b

Параметры модели

In [ ]:
model_question = llm # Имя модели, используемой для генерации ответов

temperature_question = 0 # Задаем температуру генерации ответа

verbose_question = 1 # Флаг для включения/выключения вывода ответа модели

max_tokens=1000 # Количество токенов на ответ

Запрос 1

In [ ]:
topic = "мне нужен английский язык для работы"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 1.00 секунд

Ответ Агента по выявлению вопросов:
 Когда вы готовитесь к работе, нужен ли вам английский язык?


Запрос 2

In [ ]:
topic = "Какой именно язык можно изучить"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 0.48 секунд

Ответ Агента по выявлению вопросов:
 Какой именно язык можно изучить?


Запрос 3

In [ ]:
topic = "Хочу уехать за границу летом"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 0.71 секунд

Ответ Агента по выявлению вопросов:
 Хотите ли вы уехать за границу летом?


Запрос 4

In [ ]:
topic = "Обучение проходит онлайн"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 0.52 секунд

Ответ Агента по выявлению вопросов:
 Обучение проходит онлайн.


Запрос 5

In [ ]:
topic = "Какова длительность одного занятия?"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 0.44 секунд

Ответ Агента по выявлению вопросов:
 Какова длительность одного занятия?


### Модель llama-3.1-70b

Параметры модели

In [ ]:
model_question = llm # Имя модели, используемой для генерации ответов

temperature_question = 0 # Задаем температуру генерации ответа

verbose_question = 1 # Флаг для включения/выключения вывода ответа модели

max_tokens=1000 # Количество токенов на ответ

Запрос 1

In [ ]:
topic = "мне нужен английский язык для работы"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 5.36 секунд

Ответ Агента по выявлению вопросов:
 Какой уровень английского языка необходим для работы?


Запрос 2

In [ ]:
topic = "Какой именно язык можно изучить"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 4.67 секунд

Ответ Агента по выявлению вопросов:
 Какой именно язык можно изучить?


Запрос 3

In [ ]:
topic = "Хочу уехать за границу летом"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 7.56 секунд

Ответ Агента по выявлению вопросов:
 Какие страны можно посетить летом?


Запрос 4

In [ ]:
topic = "Обучение проходит онлайн"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 6.72 секунд

Ответ Агента по выявлению вопросов:
 Какие преимущества онлайн-обучения?


Запрос 5

In [ ]:
topic = "Какова длительность одного занятия?"
user_question_llama(model_question, system_prompt_question, instructions_question, topic, temperature_question, verbose_question, max_tokens);

Время ответа: 4.33 секунд

Ответ Агента по выявлению вопросов:
 Какова длительность одного занятия?


### Модель GPT-4o-mini

Параметры модели

In [ ]:
model_question = 'gpt-4o-mini' # Имя модели, используемой для генерации ответов

temperature_question = 0 # Задаем температуру генерации ответа

verbose_question = 1 # Флаг для включения/выключения вывода ответа модели

Запрос 1

In [ ]:
topic = "мне нужен английский язык для работы"
user_question(system_prompt_question, instructions_question, topic, temperature_question, verbose_question, model_question);

Время ответа: 0.53 секунд

Ответ Агента по выявлению вопросов:
 Как мне улучшить свои знания английского языка для работы?


Запрос 2

In [ ]:
topic = "Какой именно язык можно изучить"
user_question(system_prompt_question, instructions_question, topic, temperature_question, verbose_question, model_question);

Время ответа: 0.39 секунд

Ответ Агента по выявлению вопросов:
 Какой именно язык можно изучить?


Запрос 3

In [ ]:
topic = "Хочу уехать за границу летом"
user_question(system_prompt_question, instructions_question, topic, temperature_question, verbose_question, model_question);

Время ответа: 0.49 секунд

Ответ Агента по выявлению вопросов:
 Какую страну вы планируете посетить за границей летом?


Запрос 4

In [ ]:
topic = "Обучение проходит онлайн"
user_question(system_prompt_question, instructions_question, topic, temperature_question, verbose_question, model_question);

Время ответа: 0.37 секунд

Ответ Агента по выявлению вопросов:
 Как проходит обучение?


Запрос 5

In [ ]:
topic = "Какова длительность одного занятия?"
user_question(system_prompt_question, instructions_question, topic, temperature_question, verbose_question, model_question);

Время ответа: 0.47 секунд

Ответ Агента по выявлению вопросов:
 Какова длительность одного занятия?


## Агент Презентации

In [ ]:
#@title Функция для презентации продукта

def prez_user_question(system, instructions, potr_history, search_index, temp=0, verbose=0, k=3, model= "gpt-4o"):

    knowledge_base = search_index.similarity_search(potr_history, k=k)
    docs_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n==================\n' + doc.page_content + '\n' for doc in knowledge_base]))
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}

         Отчет по потребностям: {potr_history}

         База Знаний: {docs_content}'''}
    ]
    #if verbose: print('\n==================\n')
    #if verbose: print('Потребности:\n==================\n',
                         #potr_history)
    #if verbose: print(f'База знаний:\n==================\n', docs_content)
    start_time = time.time()
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )
    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion.choices[0].message.content
    #try:
      #answer = answer.split(': ')[1]+ ' '
    #except:
      #answer = answer
    #answer = answer.lstrip('#3')
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')

    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")
    if verbose: print('\n==================')
    if verbose: print(f'Менеджер:\n',
                      f'{formatted_answer}')
    return answer

def prez_user_question_llama(model, system, instructions, potr_history, search_index, temp=0, verbose=0, k=3, max_tokens=1000):

    knowledge_base = search_index.similarity_search(potr_history, k=k)
    docs_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n==================\n' + doc.page_content + '\n' for doc in knowledge_base]))
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}

         Отчет по потребностям: {potr_history}

         База Знаний: {docs_content}'''}
    ]
    #if verbose: print('\n==================\n')
    #if verbose: print('Потребности:\n==================\n',
                         #potr_history)
    #if verbose: print(f'База знаний:\n==================\n', docs_content)

    start_time = time.time()  # Начало отсчета времени

    completion = model.create_chat_completion(  # Вызываем метод для генерации ответа
            max_tokens=max_tokens,  # Максимальное количество токенов в ответе
            temperature=temp,  # Температура генерации (0 = детерминированный ответ)
            messages=messages # Сообщения для модели
        )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени


    # Извлекаем ответ из ответа модели
    answer = completion['choices'][0]['message']['content']
    #try:
      #answer = answer.split(': ')[1]+ ' '
    #except:
      #answer = answer
    #answer = answer.lstrip('#3')
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")
    if verbose: print('\n==================')
    if verbose: print(f'Менеджер:\n',
                      f'{formatted_answer}')
    return answer

# Функция для презентации продукта с учетом теста

def prez_user_question_test(system, instructions, potr_history, search_index, otv_prov, temp=0, verbose=0, k=3, model= "gpt-4o"):

    knowledge_base = search_index.similarity_search(potr_history, k=k)
    docs_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n==================\n' + doc.page_content + '\n' for doc in knowledge_base]))
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}

         Отчет по потребностям: {potr_history}

         Оценка уровня знаний клиента: {otv_prov}

         База Знаний компании: {docs_content}'''}
    ]
    #if verbose: print('\n==================\n')
    #if verbose: print('Потребности:\n==================\n',
                         #potr_history)
    #if verbose: print(f'База знаний:\n==================\n', docs_content)

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )
    answer = completion.choices[0].message.content
    #try:
      #answer = answer.split(': ')[1]+ ' '
    #except:
      #answer = answer
    #answer = answer.lstrip('#3')
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print('\n==================')
    if verbose: print(f'Менеджер:\n',
                      f'{formatted_answer}')
    return answer

system_prompt_prez_test = '''Вы - лучший специалист по презентации продукта и компании,
занимающейся продажей курсов по английскому языку. Ваш стиль общения деловой и очень краткий.
Ваша цель: сделать краткую убедительную и качественную презентацию по потребностям клиента (запрос может быть о курсах/программах обучения, стоимости).
Ваши презентации всегда основаны на потребностях, желаниях клиента и оценке уровня знаний клиента.
Не упоминай явно потребности клиента.
При ответе используй Базу знаний компании.
Вы никогда не используете шаблонный скриптовый вариант презентации, всегда делаете её в неформальной форме.
'''

instructions_prez_test = '''
Сделайте максимально краткую убедительную и качественную презентацию, учитывая Отчет по потребностям и Оценку уровня знаний клиента. В завершении предложи оставить предзаказ на сайте.
'''
# Задаем температуру генерации ответа
# Температура 0 означает, что модель будет давать более детерминированные ответы, строго опираясь на предоставленный контекст
temperature_prez_test = 0

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks_test = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose_prez_test = 0

# Имя модели, используемой для генерации ответов
model_prez_test = "gpt-4o"

def prez_user_question_d_llama(model, system, instructions, potr_history, search_index, temp=0, verbose=0, k=3, max_tokens=1000):

    knowledge_base = search_index.similarity_search(potr_history, k=k)
    docs_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n==================\n' + doc.page_content + '\n' for doc in knowledge_base]))
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}

         Отчет по потребностям: {potr_history}

         База Знаний: {docs_content}'''}
    ]
    #if verbose: print('\n==================\n')
    #if verbose: print('Потребности:\n==================\n',
                         #potr_history)
    #if verbose: print(f'База знаний:\n==================\n', docs_content)

    start_time = time.time()  # Начало отсчета времени

    completion = model.create_chat_completion(  # Вызываем метод для генерации ответа
            max_tokens=max_tokens,  # Максимальное количество токенов в ответе
            temperature=temp,  # Температура генерации (0 = детерминированный ответ)
            messages=messages # Сообщения для модели
        )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени


    # Извлекаем ответ из ответа модели
    answer = completion['choices'][0]['message']['content']
    #try:
      #answer = answer.split(': ')[1]+ ' '
    #except:
      #answer = answer
    #answer = answer.lstrip('#3')
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    formatted_answer = textwrap.fill(answer, width=120)
    print('\n==================')
    print(f'Менеджер:\n',
                      f'{answer}')
    return answer


Промт и инструкция

In [ ]:
system_prompt_prez = '''Вы - лучший специалист по презентации продукта и компании,
занимающейся продажей курсов по английскому языку. Ваш стиль общения деловой и очень краткий.
Ваша цель: сделать краткую убедительную и качественную презентацию по потребностям клиента (запрос может быть о курсах/программах обучения, стоимости).
Ваши презентации всегда основаны на потребностях и желаниях клиента.
Не упоминай явно потребности клиента.
Вы никогда не используете шаблонный скриптовый вариант презентации, всегда делаете её в неформальной форме.
'''

instructions_prez = '''
Сделайте максимально краткую убедительную и качественную презентацию, учитывая Отчет по потребностям. В завершении предложи оставить предзаказ на сайте.
'''

### Модель llama-3.1-8b

Параметры модели

In [ ]:
# Задаем температуру генерации ответа
# Температура 0 означает, что модель будет давать более детерминированные ответы, строго опираясь на предоставленный контекст
temperature_prez = 0

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose_prez = 1

# Имя модели, используемой для генерации ответов
model_prez = llm

max_tokens=1000 # Количество токенов на ответ

Запрос 1

In [ ]:
# Список выявленных потребностей
needs = "Цель:путешествие"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 12.46 секунд

Менеджер:
 Здравствуйте!   Я знаю, что вы готовитесь к путешествию и хотите улучшить свои навыки английского языка.   Наша
программа "Take your first steps in the USA" - идеальный выбор для вас! В ней вы узнаете, как ориентироваться в
аэропорту, общаться с персоналом, справляться с таможенным досмотром и многое другое.  Курс состоит из 5 уроков, каждый
из которых поможет вам чувствовать себя комфортно в США. Вы узнаете, как снять квартиру или дом, как пользоваться
психологическими тестами, как описывать вещи и товары и многое другое.  Начните с первого урока бесплатно и проверьте
свой уровень языка. Если вы хотите продолжить обучение, мы предлагаем бонусы при оплате нескольких занятий.  Наша
программа поможет вам чувствовать себя уверенно в США и наслаждаться путешествием.   Оставьте предзаказ на сайте и
начните свой путь к совершенству! [Ссылка на сайт]


Запрос 2

In [ ]:
needs = "Цель: жить за границей"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 15.97 секунд

Менеджер:
 Здравствуйте! Я рада представить вам нашу программу обучения английскому языку, которая поможет вам достичь своих целей
и реализовать мечты.  Вы хотите жить за границей, и мы понимаем, что это требует не только языковых навыков, но и знаний
о культуре, обычалях и жизни в другой стране. Наш курс "Английский для эмиграции" поможет вам подготовиться к переезду,
ответив на все ваши вопросы и предоставив вам реальный опыт эмиграции наших учителей.  Кроме того, мы предлагаем
бесплатный курс "Самоучитель от Skyeng", который поможет вам улучшить свои языковые навыки в удобном для вас формате и в
любое время.  Наша программа обучения включает в себя:  * Созванивания с преподавателем на нашей платформе * Общение на
английском и выполнение упражнений * Домашку и следование за своим прогрессом  Мы также предлагаем бонусы при покупке
нескольких занятий и различные форматы обучения, включая ситуации, аудиокниги, видеопрактику и словарь.  Уже 103 000
учеников пр

Запрос 3

In [ ]:
needs = "Цель: работа в международной компании"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 17.26 секунд

Менеджер:
 Здравствуйте!   Я знаю, что вы стремитесь к карьере в международной компании, и для этого вам нужно улучшить свои навыки
английского языка.   Наша программа курсов делового английского включает в себя 31 онлайн-урок и 5 тематических модулей,
которые помогут вам развить языковые навыки делового общения, такие как составление резюме, ведение переписки и участие
в переговорах.   Курс делового английского будет полезен профессионалам, желающим получить работу в международной
компании.   Вы можете выбрать цель, которая вам подходит и будет вдохновлять на изучение английского. А если
сформулировать ее пока не получается, запишитесь на бесплатный вводный урок. Наш методист поможет найти подходящую цель
и настроить курс под уровень языка так, чтобы достичь ее самым эффективным путем.   Наша программа курсов делового
английского включает в себя:  * 31 онлайн-урок * 5 тематических модулей * Разработку навыков делового общения *
Улучшение навыков составления

Запрос 4

In [ ]:
needs = "Цель: Лучше всех знать английский язык"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 14.82 секунд

Менеджер:
 Здравствуйте!   Я рад представить вам нашу систему обучения английскому языку, которая поможет вам достичь своих целей и
улучшить навыки общения.  Наша система включает в себя:  *   Ситуации, аудиокниги, видеопрактику и словарь для изучения
языка в удобном формате. *   Тренировку произношения с голосовым вводом ответов и анализом произношения. *   Возможность
изучать английский в браузере или через мобильное приложение. *   Открытый бесплатный курс для глухих и слабослышащих от
Skyeng и Центра внимания.  Наша система подойдет тем, кто хочет:  *   Проводить встречи с иностранными партнерами и
общаться с людьми по всему миру. *   Получить повышение на работе, поступить в университет или открыть доступ к контенту
на языке оригинала. *   Находить единомышленников, ездить на тематические мероприятия и развивать навыки.  Наш методист
поможет вам найти подходящую цель и настроить курс под уровень языка так, чтобы достичь ее самым эффективным путем.
Хотит

Запрос 5

In [ ]:
needs = "Цель: Общение с иностранными друзьями"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 16.77 секунд

Менеджер:
 Здравствуйте!   Я рада представить вам нашу программу курсов по английскому языку для делового общения. Это 31-урочная
онлайн-программа, разделенная на 5 тематических модулей: Networking, Customer Care, Project Management, Leadership и
Career ladder.  Этот курс идеально подходит для профессионалов, желающих получить работу в международной компании или
улучшить свои навыки делового общения. Вы узнаете, как составлять резюме, вести переписку, участвовать в переговорах и
многое другое.  Наша программа курса включает в себя:  * 31 онлайн-урок * 5 тематических модулей * Уроки, которые
помогут вам начать разговор, знакомиться, находить нужные темы для беседы и многое другое  Вы также можете выбрать наш
бесплатный курс английского языка, который подойдет тем, кто хочет прокачать язык самостоятельно и реально улучшить свои
навыки.  Наше обучение доступно в различных форматах:  * Ситуации * Аудиокниги * Тренажёр слов * Видеопрактика  И уже
103 000 учеников

### Модель llama-3.1-70b

Параметры модели

In [ ]:
# Задаем температуру генерации ответа
# Температура 0 означает, что модель будет давать более детерминированные ответы, строго опираясь на предоставленный контекст
temperature_prez = 0

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose_prez = 1

# Имя модели, используемой для генерации ответов
model_prez = llm

max_tokens=1000 # Количество токенов на ответ

Запрос 1

In [ ]:
# Список выявленных потребностей
needs = "Цель:путешествие"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 80.29 секунд

Менеджер:
 Добрый день!  Если вы планируете путешествие за границу, важно чувствовать себя уверенно в общении с местными жителями.
Наш бесплатный курс английского языка поможет вам улучшить свои навыки и комфортно чувствовать себя за границей.  На
нашем курсе вы научитесь:  * Рассказывать об отпуске и узнавать про это у других * Читать городские знаки и узнавать
путь * Держать себя в руках и понимать правила разных стран * Купить билет на автобус, самолет, пароход или любой другой
вид транспорта  Кроме того, на нашем курсе вы узнаете о особенностях въезда в США, типах жилья, популярных на вечеринках
играх и многом другом.  Пройдите первый урок бесплатно и познакомьтесь с нашей платформой. Если вы решите продолжить
обучение, мы предложим вам бонусы при оплате нескольких занятий.  Оставьте предзаказ на нашем сайте и начните свой путь
к уверенному общению на английском языке!


Запрос 2

In [ ]:
needs = "Цель: жить за границей"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 118.52 секунд

Менеджер:
 Добрый день!  Если вы мечтаете жить за границей, то знание английского языка - это ключ к вашей новой жизни. Мы
предлагаем вам бесплатный курс английского языка, который поможет вам улучшить свои навыки и подготовиться к переезду.
Наш курс включает в себя:  * Полезные диалоги в посольстве, банке, страховой компании и других местах, где вы будете
общаться на английском языке * Чек-листы по жизни за границей, которые помогут вам подготовиться к переезду * Реальный
опыт эмиграции наших учителей, которые поделятся с вами своими историями и советами  Курс ответит на такие вопросы, как:
* Как выбрать страну для эмиграции? * Как получить визу и пройти интервью в посольстве на английском языке? * Как найти
жилье и перевезти свои вещи? * Как устроить ребенка в детский сад? * Как начать бизнес и открыть счет в банке?  Обучение
проходит в удобном для вас формате: вы созваниваетесь с преподавателем на нашей платформе, общаетесь на английском и
выполняете упр

Запрос 3

In [ ]:
needs = "Цель: работа в международной компании"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 99.54 секунд

Менеджер:
 Добрый день!  Если вы стремитесь к карьерному росту в международной компании, то знание английского языка является
ключевым фактором успеха. Наш курс делового английского предназначен именно для таких целеустремленных людей, как вы.  В
нашей программе вы найдете 31 онлайн-урок, разделенных на 5 тематических модулей: Networking, Customer Care, Project
Management, Leadership и Career Ladder. Мы поможем вам развить навыки делового общения, необходимые для работы в
международной компании.  Вы узнаете, как вести small talk, знакомиться с бизнес-партнерами, эффективно использовать
конференции и многое другое. Наши уроки позволят вам начинать разговор, знакомиться, находить нужные темы для беседы и
избегать острых тем.  Кроме того, наш курс технического английского для IT-специалистов поможет вам общаться с
зарубежными коллегами, читать техническую документацию, разбираться в интерфейсе ПО и многое другое.  Если вы готовы
сделать шаг вперед в своей карье

Запрос 4

In [ ]:
needs = "Цель: Лучше всех знать английский язык"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 72.80 секунд

Менеджер:
 Добрый день!  Мы предлагаем уникальные курсы по английскому языку, которые помогут вам достичь ваших целей и улучшить
вашу карьеру. Наше обучение доступно в различных форматах, включая ситуации, аудиокниги, тренажёр слов и видеопрактику.
Более 103 000 учеников уже улучшили свой английский с нами. Мы предлагаем тренировку произношения с голосовым вводом
ответов и анализом произношения.  Наши курсы доступны в браузере и через мобильное приложение, что позволяет вам изучать
английский в любое время и в любом месте.  Мы также предлагаем бесплатный вводный урок, который поможет вам найти
подходящую цель и настроить курс под ваш уровень языка.  Выберите свою цель и начните изучать английский с нами уже
сегодня!  Оставьте предзаказ на нашем сайте и получите доступ к нашим уникальным курсам и ресурсам.


Запрос 5

In [ ]:
needs = "Цель: Общение с иностранными друзьями"
prez_user_question_llama(model_prez, system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, max_tokens);

Время ответа: 76.78 секунд

Менеджер:
 Добрый день!  Если вы хотите общаться с иностранными друзьями, то наш курс английского языка для делового общения -
идеальный выбор для вас.   В нашем курсе вы научитесь:  * Ведению small talk и стратегиям знакомства с бизнес-партнерами
* Эффективному использованию конференций и деловой переписке * Нахождению нужных тем для беседы и избеганию острых тем *
Поддержанию контакта после знакомства  Курс состоит из 31 онлайн-урока и 5 тематических модулей: Networking, Customer
Care, Project Management, Leadership и Career ladder.  Помимо этого, у нас есть бесплатный курс английского, который
подойдет тем, кто хочет прокачать язык самостоятельно. В нем вы найдете авторские материалы и задания для практики в
удобном формате.  Если вы хотите улучшить свои навыки английского языка для общения с иностранными друзьями, то оставьте
предзаказ на нашем сайте и начните обучение уже сегодня!


### Модель GPT-4o-mini

Параметры модели

In [ ]:
# Задаем температуру генерации ответа
# Температура 0 означает, что модель будет давать более детерминированные ответы, строго опираясь на предоставленный контекст
temperature_prez = 0

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose_prez = 1

# Имя модели, используемой для генерации ответов
model_prez = "gpt-4o-mini"

Запрос 1

In [ ]:
# Список выявленных потребностей
needs = "Цель:путешествие"
prez_user_question(system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, model= model_prez);

Время ответа: 2.33 секунд

Менеджер:
 Добрый день!  Если ваша цель — путешествия, мы предлагаем идеальное решение для комфортного общения за границей. Наши
курсы английского языка помогут вам не только улучшить навыки, но и уверенно чувствовать себя в любой стране.  Вы
сможете: - Легко общаться с местными жителями. - Ориентироваться в аэропортах и на вокзалах. - Понимать культурные
нюансы и наслаждаться искусством в оригинале.  Начните с бесплатного урока, чтобы оценить качество обучения и
познакомиться с нашей платформой.   Не упустите возможность сделать первый шаг к вашим путешествиям! Оставьте предзаказ
на нашем сайте и начните обучение уже сегодня.


Запрос 2

In [ ]:
needs = "Цель: жить за границей"
prez_user_question(system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, model= model_prez);

Время ответа: 2.57 секунд

Менеджер:
 Добрый день!  Если ваша цель — жизнь за границей, мы можем помочь вам достичь её с помощью наших курсов английского
языка. Мы предлагаем уникальные программы, которые охватывают все аспекты, необходимые для комфортной жизни в другой
стране.  Наши курсы включают: - Практические диалоги для общения в посольствах, банках и других учреждениях. - Полезные
советы по эмиграции, включая выбор страны, получение визы и адаптацию на новом месте. - Разнообразные форматы обучения:
от ситуационных упражнений до видеопрактики, что позволяет учиться в удобное время и в любом месте.  С нами вы сможете
не только улучшить язык, но и открыть новые горизонты для карьеры, общения и культурного обогащения.  Не упустите
возможность! Оставьте предзаказ на нашем сайте и начните свой путь к новой жизни уже сегодня.


Запрос 3

In [ ]:
needs = "Цель: работа в международной компании"
prez_user_question(system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, model= model_prez);

Время ответа: 3.32 секунд

Менеджер:
 Добрый день!  Если ваша цель — работа в международной компании, мы предлагаем курсы, которые помогут вам достичь этого.
Наша программа делового английского включает 31 онлайн-урок и 5 тематических модулей, охватывающих ключевые навыки,
такие как деловая переписка, ведение переговоров и networking.   Вы сможете не только улучшить свои языковые навыки, но
и научиться эффективно общаться с зарубежными коллегами, что откроет новые карьерные возможности.   Также у нас есть
бесплатный самоучитель, который позволит вам прокачать язык самостоятельно в удобном формате.  Если вы готовы сделать
шаг к своей цели, оставьте предзаказ на нашем сайте. Это ваш шанс начать обучение уже сегодня!


Запрос 4

In [ ]:
needs = "Цель: Лучше всех знать английский язык"
prez_user_question(system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, model= model_prez);

Время ответа: 2.33 секунд

Менеджер:
 Добрый день!  Мы предлагаем уникальные курсы английского языка, которые помогут вам достичь ваших целей. Наша программа
охватывает все аспекты языка: от общения с иностранными партнерами до чтения оригинальных произведений искусства.   Вы
сможете учиться в удобном формате: через браузер или мобильное приложение, а также использовать разнообразные материалы
— от аудиокниг до видеопрактики. Более 103 000 учеников уже выбрали нас для улучшения своих навыков.  Если вы хотите
прокачать английский самостоятельно, у нас есть бесплатный самоучитель с авторскими заданиями.   Не упустите
возможность! Оставьте предзаказ на нашем сайте и начните свой путь к свободному владению английским языком.


Запрос 5

In [ ]:
needs = "Цель: Общение с иностранными друзьями"
prez_user_question(system_prompt_prez, instructions_prez, needs, db, temperature_prez, verbose_prez, relevant_chanks, model= model_prez);

Время ответа: 2.24 секунд

Менеджер:
 Мы предлагаем курс английского языка, который идеально подходит для общения с иностранными друзьями и расширения ваших
горизонтов. Программа включает 31 онлайн-урок, охватывающий ключевые аспекты делового общения, такие как networking,
customer care и leadership.   Вы сможете уверенно вести беседы, знакомиться с новыми людьми и поддерживать контакты, а
также развивать навыки, необходимые для карьерного роста и международных поездок.   Обучение доступно в удобном формате:
на компьютере, планшете или через мобильное приложение.   Не упустите возможность улучшить свои навыки! Оставьте
предзаказ на нашем сайте и начните обучение уже сегодня.


# Диалог

In [ ]:
#@title Запуск диалога Llama-3.1-8b
# Инициализация списков для хранения истории чата, вопросов клиента и ответов менеджера
history_chat = []
history_user = []
history_manager = []
history_potr = []

# Вывод ответов агентов
verbose_router = 0
verbose = 0
verbose_question = 0
verbose_prez = 0
verbose_stilist = 0

model = llm

# Приветственное сообщение
welcome_message = """Здравствуйте, я нейро-консультант школы английского языка Skyeng. Чем я могу Вам помочь? """

# Добавление приветственного сообщения в историю чата и ответов
history_chat.append(f"Менеджер: {welcome_message}")
history_manager.append(welcome_message)
print(welcome_message)

# Бесконечный цикл для общения с клиентом
first_question = True

first_question_2 = True

keywords_matched = False  # Флаг для отслеживания совпадения ключевых слов

while True:
    print()
    # Получение вопроса от клиента
    client_question = input('Вопрос клиента:')

    # Добавление вопроса клиента в историю вопросов и чата
    history_user.append(client_question)
    history_chat.append(f"Клиент: {client_question}")

    # Проверка, если клиент ввел 'stop' или 'стоп', прерываем цикл
    if client_question.lower() in ['stop', 'стоп']:
        break

    # Генерация ответа от роутера на вопрос клиента
    if first_question:
        router_input = f"Менеджер: {history_manager[-1]} Клиент: {history_user[-1]}"
        first_question = False
    else:
        router_input = f"Менеджер: {answer_2} Клиент: {client_question}"


    router_response = user_question_router_llama(model=model,
                                                 system=system_prompt_router,
                                           instructions=instructions_router,
                                           topic=router_input,
                                           temp=temperature_router, verbose=verbose_router,
                                           max_tokens=1000)


    # Добавление ответа роутера в историю потребностей
    history_potr.append(router_response)

    #print("Отчет о потребностях: ", history_potr)
    # Проверка результата роутера

    if len(history_potr)>3: # Когда сделана презентация и выявлены потребности - свободные ответы на вопросы
        #print("Ветка 2")
        if keywords_matched == False:
            answers_test = user_potr_llama(model, history_potr, temp=0, verbose=0, max_tokens=1000)
            #print("Основная потребность:", answers_test)
                # Генерация ответа менеджера с помощью prez_user_question
            answer_prez = prez_user_question_d_llama(model, system_prompt_prez, instructions_prez, answers_test, db, temp=temperature_prez, verbose=1, k=relevant_chanks, max_tokens=1000)
                # Стилизация объединенного ответа с использованием функции stilizator_answer
            #answer = stilizator_answer_d_llama(model, system_prompt_stilist_prez, instructions_stilist, answer_prez, temperature_stilist, verbose=1, max_tokens=1000)

            test_mes = """Что бы Вы хотели еще узнать? """
            print(test_mes)

            keywords_matched = True  # Устанавливаем флаг, чтобы больше не входить в эту ветку

        else:

            answer = answer_kons_d_llama(model, system, instruction, client_question, db, history_chat, temperature, verbose, relevant_chanks, max_tokens=1000)
            print(answer)

        #print(answer)


    else: # Когда не выявлены потребности
        #print("Ветка 3")
        # Генерация ответа от роутера на вопрос клиента
        if first_question_2:
            answer_vopros = user_question_llama(model, system_prompt_question, instructions_question, client_question, temp=temperature_question, verbose=verbose_question, max_tokens=1000)
            first_question_2 = False
        else:
            client_question_2 = answer_2 + client_question
            answer_vopros = user_question_llama(model, system_prompt_question, instructions_question, client_question_2, temp=temperature_question, verbose=verbose_question, max_tokens=1000)
        # Генерация ответа менеджера с использованием функций answer_kons и spez_user_question
        answer_1 = answer_kons_d_llama(model, system, instruction, answer_vopros, db, history_chat, temperature, verbose, relevant_chanks, max_tokens=1000)
        answer_2 = spez_user_question_llama(model, system_prompt_potr, instructions__potr, router_response, history_chat, temp, verbose=0, max_tokens=1000)

        # Объединение первого и второго ответов менеджера
        #answer_stilist = answer_1 + answer_2
        # Стилизация объединенного ответа с использованием функции stilizator_answer
        #answer_stilist = stilizator_answer_d_llama(model, system_prompt_stilist, instructions_stilist, answer_1, temperature_stilist, verbose=1, max_tokens=1000)
        answer = answer_1 +"\n"+ answer_2
        print(answer)

    # Добавление ответа менеджера в историю чата и ответов
    history_chat.append(f"Менеджер: {answer}")
    history_manager.append(answer)




Здравствуйте, я нейро-консультант школы английского языка Skyeng. Чем я могу Вам помочь? 

Вопрос клиента:как у вас происходит обучение?

Менеджер:
Мы подбираем план обучения под потребности и цели каждого ученика, используя проверенную коммуникативную методологию и систему контроля прогресса. Это позволяет ученикам начать использовать язык уже с первого занятия.
Как вы думаете, что могло бы помочь вам сохранять мотивацию и интерес к изучению английского языка?

Вопрос клиента:я вспоминаю конечную цель и это меня вдохновляет

Менеджер:
Я думаю, что помочь сохранять мотивацию и интерес к изучению английского языка могут следующие факторы:

- Установление четкой цели и плана обучения.
- Регулярные достижения и прогресс в изучении языка.
- Возможность выбора интересных и актуальных тем для изучения.
- Поддержка и мотивация от преподавателя и сообщества учеников.
- Использование различных методов обучения и материалов, чтобы избежать монотонности.

Также, как вы упомянули, вспоминание коне

In [ ]:
#@title Запуск диалога Llama-3.1-70b
# Инициализация списков для хранения истории чата, вопросов клиента и ответов менеджера
history_chat = []
history_user = []
history_manager = []
history_potr = []

# Вывод ответов агентов
verbose_router = 0
verbose = 0
verbose_question = 0
verbose_prez = 0
verbose_stilist = 0

model = llm

# Приветственное сообщение
welcome_message = """Здравствуйте, я нейро-консультант школы английского языка Skyeng. Чем я могу Вам помочь? """

# Добавление приветственного сообщения в историю чата и ответов
history_chat.append(f"Менеджер: {welcome_message}")
history_manager.append(welcome_message)
print(welcome_message)

# Бесконечный цикл для общения с клиентом
first_question = True

first_question_2 = True

keywords_matched = False  # Флаг для отслеживания совпадения ключевых слов

while True:
    print()
    # Получение вопроса от клиента
    client_question = input('Вопрос клиента:')

    # Добавление вопроса клиента в историю вопросов и чата
    history_user.append(client_question)
    history_chat.append(f"Клиент: {client_question}")

    # Проверка, если клиент ввел 'stop' или 'стоп', прерываем цикл
    if client_question.lower() in ['stop', 'стоп']:
        break

    # Генерация ответа от роутера на вопрос клиента
    if first_question:
        router_input = f"Менеджер: {history_manager[-1]} Клиент: {history_user[-1]}"
        first_question = False
    else:
        router_input = f"Менеджер: {answer_2} Клиент: {client_question}"


    router_response = user_question_router_llama(model=model,
                                                 system=system_prompt_router,
                                           instructions=instructions_router,
                                           topic=router_input,
                                           temp=temperature_router, verbose=verbose_router,
                                           max_tokens=1000)


    # Добавление ответа роутера в историю потребностей
    history_potr.append(router_response)

    #print("Отчет о потребностях: ", history_potr)
    # Проверка результата роутера

    if len(history_potr)>3: # Когда сделана презентация и выявлены потребности - свободные ответы на вопросы
        #print("Ветка 2")
        if keywords_matched == False:
            answers_test = user_potr_llama(model, history_potr, temp=0, verbose=0, max_tokens=1000)
            #print("Основная потребность:", answers_test)
                # Генерация ответа менеджера с помощью prez_user_question
            answer_prez = prez_user_question_d_llama(model, system_prompt_prez, instructions_prez, answers_test, db, temp=temperature_prez, verbose=1, k=relevant_chanks, max_tokens=1000)
                # Стилизация объединенного ответа с использованием функции stilizator_answer
            #answer = stilizator_answer_d_llama(model, system_prompt_stilist_prez, instructions_stilist, answer_prez, temperature_stilist, verbose=1, max_tokens=1000)

            test_mes = """Что бы Вы хотели еще узнать? """
            print(test_mes)

            keywords_matched = True  # Устанавливаем флаг, чтобы больше не входить в эту ветку

        else:

            answer = answer_kons_d_llama(model, system, instruction, client_question, db, history_chat, temperature, verbose, relevant_chanks, max_tokens=1000)
            print(answer)

        #print(answer)


    else: # Когда не выявлены потребности
        #print("Ветка 3")
        # Генерация ответа от роутера на вопрос клиента
        if first_question_2:
            answer_vopros = user_question_llama(model, system_prompt_question, instructions_question, client_question, temp=temperature_question, verbose=verbose_question, max_tokens=1000)
            first_question_2 = False
        else:
            client_question_2 = answer_2 + client_question
            answer_vopros = user_question_llama(model, system_prompt_question, instructions_question, client_question_2, temp=temperature_question, verbose=verbose_question, max_tokens=1000)
        # Генерация ответа менеджера с использованием функций answer_kons и spez_user_question
        answer_1 = answer_kons_d_llama(model, system, instruction, answer_vopros, db, history_chat, temperature, verbose, relevant_chanks, max_tokens=1000)
        answer_2 = spez_user_question_llama(model, system_prompt_potr, instructions__potr, router_response, history_chat, temp, verbose=0, max_tokens=1000)

        # Объединение первого и второго ответов менеджера
        #answer_stilist = answer_1 + answer_2
        # Стилизация объединенного ответа с использованием функции stilizator_answer
        #answer_stilist = stilizator_answer_d_llama(model, system_prompt_stilist, instructions_stilist, answer_1, temperature_stilist, verbose=1, max_tokens=1000)
        answer = answer_1 +"\n"+ answer_2
        print(answer)

    # Добавление ответа менеджера в историю чата и ответов
    history_chat.append(f"Менеджер: {answer}")
    history_manager.append(answer)




Здравствуйте, я нейро-консультант школы английского языка Skyeng. Чем я могу Вам помочь? 

Вопрос клиента:как у вас происходит обучение?

Менеджер:
Мы подбираем план обучения под потребности и цели каждого ученика и, самое главное, доводим до результата с помощью проверенной коммуникативной методологии и системы контроля прогресса.
Вы когда-нибудь думали о том, чтобы улучшить свой английский язык для личного развития или хобби?

Вопрос клиента:нет, я хочу работать в иностранной IT компании

Менеджер:
Чтобы улучшить свой английский язык для работы в иностранной IT компании, можно начать с изучения технического английского языка, который включает в себя уроки по коммуникации и работе в команде. Это поможет вам научиться общаться с зарубежными коллегами, читать техническую документацию, разбираться в интерфейсе ПО и заниматься программированием сайтов и других продуктов.
Вы планируете работать в конкретной стране или готовы рассматривать предложения из разных стран?

Вопрос клиента:я хочу